In [ ]:
from datetime import date
from itertools import product

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

from app.model import DispatchConfig
from run_dispatch import run_dispatch

In [ ]:
DATES = pd.date_range(
    start="2024-06-15",
    end="2024-06-16",
    # end="2024-06-30",
    inclusive="both"
)
CONFIGS = [
    "preideal",
    "bess_preideal"
]

BESS_VALUES = {
    "BESS_1": {
        "MWh_nom": 2000,
        "hours_to_deplete": 2,
        "efficiency": 0.9,
        "min_soc": 0.05,
        "max_soc": 0.98,
        "initial_soc": 0.05,
        "charge_bid": 1E6,
        "discharge_bid": 0,
    },
}


In [ ]:

output_unstacked = pd.DataFrame()
for (date_, dispatch_type) in product(DATES,CONFIGS):
    BESS = BESS_VALUES if "bess" in dispatch_type.lower() else None
    mpo = run_dispatch(DispatchConfig(dispatch_type=dispatch_type), DISPATCH_DATE=date_.date(), show_figs=False, BESS=BESS)
    # mpo["dispatch_type"] = dispatch_typex
    output_unstacked = pd.concat([output_unstacked, mpo])
output = output_unstacked.stack().reset_index()
output.columns = ["datetime", "source", "MPO"]
output.drop_duplicates(inplace=True)

In [ ]:

fig = px.line(output, x="datetime", y="MPO", color="source", title="MPO")
fig.update_layout(
    xaxis_title="Hora",
    yaxis_title="Precio [COP/MWh]",
    # width=800,
    # height=600,
    # xaxis=dict(
    #     dtick=3_600_000,
    # ),
)
fig.show()

In [ ]:
output_unstacked = output.pivot(index="datetime", columns="source", values="MPO")

In [ ]:
# Group by days and apply the mean_absolute_percentage_error function
grouped = output_unstacked.groupby(output_unstacked.index.date).apply(
    lambda x: mean_absolute_percentage_error(y_pred=x["MPO bess_preideal Modelo"], y_true=x["MPO preideal Modelo"])
)

# Convert the result to a DataFrame for better readability
grouped_df = pd.DataFrame(grouped, columns=["MAPE %"])
grouped_df["MAPE %"] = grouped_df["MAPE %"] * 100
grouped_df

In [ ]:
# Calculate the difference between the two columns
output_unstacked['Difference'] = output_unstacked['MPO preideal Modelo'] - output_unstacked['MPO bess_preideal Modelo']

# Create an area plot with Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=output_unstacked.index,
    y=output_unstacked['Difference'],
    fill='tozeroy',
    mode='none'  # No markers or lines
))

fig.update_layout(
    title='Difference between MPO preideal Modelo and MPO bess_preideal Modelo',
    xaxis_title='Datetime',
    yaxis_title='Difference [COP/MWh]'
)

fig.show()

In [ ]:
fig = go.Figure()

# Add line for "MPO preideal Modelo"
fig.add_trace(go.Scatter(
    x=output_unstacked.index,
    y=output_unstacked['MPO preideal Modelo'],
    mode='lines',
    name='MPO preideal Modelo'
))

# Add line for "MPO bess_preideal Modelo"
fig.add_trace(go.Scatter(
    x=output_unstacked.index,
    y=output_unstacked['MPO bess_preideal Modelo'],
    mode='lines',
    name='MPO bess_preideal Modelo',
    fill='tonexty',
    line_color='green'
))

# Add area for positive differences
# fig.add_trace(go.Scatter(
#     x=output_unstacked.index,
#     y=output_unstacked['Difference'].apply(lambda x: x if x > 0 else 0),
#     fill='tozeroy',
#     mode='none',
#     fillcolor='rgba(0, 255, 0, 0.5)',  # Green color for positive differences
#     name='Positive Difference',
#     line_shape='hv',
#     yaxis='y2'
# ))

# # Add area for negative differences
# fig.add_trace(go.Scatter(
#     x=output_unstacked.index,
#     y=output_unstacked['Difference'].apply(lambda x: x if x < 0 else 0),
#     fill='tozeroy',
#     mode='none',
#     fillcolor='rgba(255, 0, 0, 0.5)',  # Red color for negative differences
#     name='Negative Difference',
#     line_shape='hv',
#     yaxis='y2'
# ))

fig.update_layout(
    title='MPO preideal Modelo vs MPO bess_preideal Modelo',
    xaxis_title='Datetime',
    yaxis_title='MPO [COP/MWh]',
    yaxis2=dict(
        title='Difference [COP/MWh]',
        overlaying='y',
        side='right'
    ),
    legend_title='Legend'
)

fig.show()

In [ ]:
output_unstacked.columns

In [ ]:
import plotly.graph_objects as go
import numpy as np

import pandas as pd
from datetime import datetime
pd.options.plotting.backend = "plotly"

# # sample data
# df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')
# df.index = df.Date
# df = df[['AAPL.Close', 'mavg']]
# df['mavg2'] = df['AAPL.Close'].rolling(window=50).mean()
# df.columns = ['y', 'ma1', 'ma2']
# df=df.tail(250).dropna()
# df1 = df.copy()
df = output_unstacked.copy()

# split data into chunks where averages cross each other
df['label'] = np.where(df['MPO preideal Modelo']>df['MPO bess_preideal Modelo'], 1, 0)
df['group'] = df['label'].ne(df['label'].shift()).cumsum()
df = df.groupby('group')
dfs = []
for name, data in df:
    dfs.append(data)

# custom function to set fill color
def fillcol(label):
    if label >= 1:
        return 'rgba(0,250,0,0.4)'
    else:
        return 'rgba(250,0,0,0.4)'

fig = go.Figure()

# Add line for "MPO preideal Modelo"
fig.add_trace(go.Scatter(
    x=output_unstacked.index,
    y=output_unstacked['MPO preideal Modelo'],
    mode='lines',
    name='MPO preideal Modelo'
))

# Add line for "MPO bess_preideal Modelo"
fig.add_trace(go.Scatter(
    x=output_unstacked.index,
    y=output_unstacked['MPO bess_preideal Modelo'],
    mode='lines',
    name='MPO bess_preideal Modelo',
    fill='tonexty',
))



for df in dfs:
    fig.add_traces(go.Scatter(x=df.index, y = df["MPO preideal Modelo"],
                              line = dict(color='rgba(0,0,0,0)')))
    
    fig.add_traces(go.Scatter(x=df.index, y = df["MPO bess_preideal Modelo"],
                              line = dict(color='rgba(0,0,0,0)'),
                              fill='tonexty', 
                              fillcolor = fillcol(df['label'].iloc[0])))

# # include averages
# fig.add_traces(go.Scatter(x=df1.index, y = df1.ma1,
#                           line = dict(color = 'blue', width=1)))

# fig.add_traces(go.Scatter(x=df1.index, y = df1.ma2,
#                           line = dict(color = 'red', width=1)))

# # include main time-series
# fig.add_traces(go.Scatter(x=df1.index, y = df1.y,
#                           line = dict(color = 'black', width=2)))


fig.update_layout(showlegend=True)
fig.show()